In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import selenium
from urllib.parse import  urlparse
import time
import lxml
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
import re
import pickle
from contextlib import suppress

In [10]:
# 네이버 카페 게시판의 게시물을 수집하는 크롤러(제목, 본문, 등록일)

In [19]:
# 기본 정보
keyword = "갤럭시북"
cafe_code = "s9s9s9s9" #str # cafe.naver.com/cafecode
board_url = "https://cafe.naver.com/ArticleList.nhn?search.clubid=17779760&search.menuid=588&search.boardtype=L&search.totalCount=119&search.cafeId=17779760" # 게시판 url에서 page정보를 제외
page_range = 1  #int
post_num = []
contents_list = []


In [20]:
# 대상 게시판의 지정된 페이지만큼 글 번호만 추출 (리스트화)

driver = None
driver = webdriver.Chrome("chromedriver")

driver.get("https://nid.naver.com/nidlogin.login")
time.sleep(6)

for i in range(page_range):
    url = board_url + "&search.page=" + str(i+1)  # 주소 지정
        
    driver.get(url)
    
    time.sleep(1) # implicity_wait, 암묵적, 파싱 대기시간. / #time.sleep은 무조건 대기
    
    driver.switch_to.frame('cafe_main')
    
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
    a = soup.find_all('div', {'class': 'board-number'})
    
    for n in a:
        post_num.append(re.search("[0-9]{2,8}", str(n)).group())

print(post_num)
print(len(post_num))
        
# 대상 게시물의 제목, 본문, 날짜 수집

temp_title = []
temp_content = []
temp_date = []
    
for i in post_num:
    url = "https://cafe.naver.com/" + cafe_code + "/" + i
    
    driver.get(url)
    
    time.sleep(1)
        
    driver.switch_to.frame('cafe_main')

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    content = soup.find('div', class_='ArticleContentBox')    

    with suppress(AttributeError):   # 제목 없는 게시글
        temp_title.append(content.find('h3',class_='title_text').text.strip()) 
        temp_content.append(content.find("div", class_="article_viewer").text.strip())
        temp_date.append(content.find('div',class_='article_info').find('span',class_='date').text.strip())
        
driver.quit()


['65932', '65666', '65382', '65288', '65248', '65192', '65161', '64453', '63554', '63378', '63328', '63327', '62972', '58953', '58809']
15


In [21]:
# 내용 확인
arr = np.vstack([np.array(temp_title), np.array(temp_content), np.array(temp_date)])
df = pd.DataFrame(arr).T
# df
df.to_csv("C:/690009/TP2/final_project/member/NTJ/crawlerNsReview/data/" + keyword + ".csv")

In [22]:
df = pd.read_csv("C:/690009/TP2/final_project/member/NTJ/crawlerNsReview/data/" + keyword + ".csv", encoding="utf-8", names=["제목", "본문", "등록일"], header=1)
df

,제목,본문,등록일
1,삼성 갤럭시 북 NT750XDZ-A51A 후기,#후기이벤트 #이벤트참여​갤럭시북 프로 또는 갤럭시북 프로 360의 가격이 부담스러...,2021.06.25. 23:09
2,삼성 북 플렉스 2,제 개인적인 경험입니다. 최악​북플렉스 구매후 2주만에 심각한 초기불량으로 환불.....,2021.05.31. 20:48
3,갤북 프로 360 단점 위주 사용후기,바로 본론으로 들어갑니다. ​클린설치시 외부 디바이스 키보드 마우스 없으면 설치 못...,2021.05.22. 23:37
4,갤럭시북 프로 360 15.6인치 구매후기,금번 사전예약 구매기간에 갤럭시북 프로 360 15.6인치 모델을 구입했습니다. ​...,2021.05.20. 15:02
5,갤럭시 북 프로 13인치 노트북 추천,"여행가라면 선택해야 할 노트북, 갤럭시 북 프로 13인치!주로 외부 사용을 목적으로...",2021.05.17. 15:07
6,삼성 갤럭시 북 프로 !!,삼성노트북 RV520쓰다가 고장이나버려서 큰맘먹고 최신노트북 살려고 찾아보다가 삼성...,2021.05.15. 16:57
7,이온2 i3 소음 심해요,이온 2이고 i3입니다.소음이 심하네요. 독서실이나 도서관에서 인강 듣고 싶은데 백...,2021.03.21. 00:43
8,삼성 갤럭시 북 이온2 구매후기!!!!,"평소에 들고 다니는 노트북이 오래되가지고, 느리고 잘 돌아가지도 않아서~~~ 새롭게...",2021.01.28. 16:46
9,삼성 갤럭시북 이온2 (NT950XDZ-G58AW) 구매&사용후기 입니다! 노트북추천!,노트북을 정말 오랜만에 구매했습니다.(만으로 10년??정도만에 구매한듯요)​사용용도...,2021.01.19. 22:12
10,"갤럭시북 이온2 NT950XDZ-G58AW (MX450버전) 사용기, 노트북추천",삼성전자 갤럭시북 이온2 NT950XDZ-G58AW(MX450버전)을 갤럭시 아카데...,2021.01.17. 22:44
